In [1]:
import numpy as np
import tensorflow as tf


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [40]:

# get data from train_1.csv
tmp = np.genfromtxt("train_2.csv", dtype=float, delimiter=",")

# set training_data quantity and validation data quantity
train_quantity = 800
validation_quantity = 891-train_quantity

# 归一化
tem = tmp[1:,3:]
ave = np.mean(tem,axis = 0)*np.ones((891,7))
ptp = np.ptp(tem,axis = 0)*np.ones((891,7))
tem = np.subtract(tem,ave)
tem = np.true_divide(tem,ptp)
tmp[1:,3:] = tem

#apart train,validation
train_data = tmp[1:train_quantity+1,3:]
validation_data0 = tmp[train_quantity+1:,3:]

train_lable = tmp[1:train_quantity+1,2]
validation_lable0 = tmp[train_quantity+1:,2]

train_lable = train_lable.reshape(train_quantity,1)
validation_lable0 = validation_lable0.reshape(validation_quantity,1)

# define layer function
def add_layer(input,in_size,out_size):
    theta = tf.Variable(tf.random_normal([in_size,out_size]))
    if out_size==1:
        z = tf.matmul(input,theta)
        a = tf.sigmoid(z)
    else:
        z = tf.matmul(input,theta)
        a = tf.sigmoid(z)
        a_0 = tf.ones([train_quantity,1])
        a = tf.concat([a_0,a],1)
    return theta,a,z


# define placeholder for inputs to network
xs = tf.placeholder(tf.float32,[None,7])
ys = tf.placeholder(tf.float32,[None,1])

# add hidden layer
theta_1,l1,z_1 = add_layer(xs,7,12)
# add output layer
theta_2,output,z_2 = add_layer(l1,13,1)

#regularzation
lamda = 0.08
regularzation = (tf.reduce_sum(tf.square(theta_1))+tf.reduce_sum(tf.square(theta_2)))*lamda/600

# the error between prediction and real data
J_theta = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = ys,logits = z_2)) + regularzation

a =  3
# learning rate
train_step = tf.train.GradientDescentOptimizer(a).minimize(J_theta)

#prediction
validation_data = tf.constant(validation_data0,tf.float32)
validation_lable = tf.constant(validation_lable0,tf.float32)

a_1 = tf.sigmoid(tf.matmul(validation_data,theta_1))
a_0 = tf.ones([validation_quantity,1])
a_20 = tf.concat([a_0,a_1],1)
a_2 = tf.sigmoid(tf.matmul(a_20,theta_2))
prediction = tf.round(a_2)

validation = tf.subtract(prediction,validation_lable)
validation = tf.abs(validation)
accuracy =tf.reduce_sum(validation) 
accuracy = (validation_quantity-accuracy)/validation_quantity

#initialize
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for i in range(70000):
    #training
    sess.run(train_step,feed_dict={xs:train_data,ys:train_lable})
    
    if i%1000 == 0:
        loss = sess.run(J_theta,feed_dict={xs:train_data,ys:train_lable})
        print("J_theta:",loss)
        acc = sess.run(accuracy)
        print(acc)
        




J_theta: 1.287285
0.62637365
J_theta: 0.44737282
0.85714287
J_theta: 0.4416407
0.84615386
J_theta: 0.44014916
0.8681319
J_theta: 0.4398782
0.8681319
J_theta: 0.43979388
0.8681319
J_theta: 0.4397517
0.8681319
J_theta: 0.43972638
0.8681319
J_theta: 0.43971032
0.8681319
J_theta: 0.43970016
0.8681319
J_theta: 0.43969372
0.8681319
J_theta: 0.4396897
0.8681319
J_theta: 0.4396872
0.8681319
J_theta: 0.4396857
0.8681319
J_theta: 0.43968475
0.8681319
J_theta: 0.43968424
0.8681319
J_theta: 0.439684
0.8681319
J_theta: 0.43968374
0.8681319
J_theta: 0.43968362
0.8681319
J_theta: 0.43968353
0.8681319
J_theta: 0.4396835
0.8681319
J_theta: 0.43968344
0.8681319
J_theta: 0.43968347
0.8681319
J_theta: 0.43968347
0.8681319
J_theta: 0.43968344
0.8681319
J_theta: 0.4396834
0.8681319
J_theta: 0.43968344
0.8681319
J_theta: 0.43968344
0.8681319
J_theta: 0.43968344
0.8681319
J_theta: 0.43968347
0.8681319
J_theta: 0.43968344
0.8681319
J_theta: 0.43968344
0.8681319
J_theta: 0.43968344
0.8681319
J_theta: 0.43968344

In [41]:
# import test sets
test = np.genfromtxt("test_2.csv",dtype = float,delimiter=",")

#reshape test sets
tem1 = test[1:,2:]
ave1 = np.mean(tem1,axis = 0)*np.ones((418,7))
ptp1 = np.ptp(tem1,axis = 0)*np.ones((418,7))
tem1 = np.subtract(tem1,ave1)
tem1 = np.true_divide(tem1,ptp1)
test[1:,2:] = tem1
test = test[1:,2:]

# define tensorflow constant test data
test_data = tf.constant(test,tf.float32)

# predict test data
t_1 = tf.sigmoid(tf.matmul(test_data,theta_1))
t_0 = tf.ones([418,1])
t_20 = tf.concat([t_0,t_1],1)
t_2 = tf.sigmoid(tf.matmul(t_20,theta_2))
prediction_1 = tf.round(t_2)
prediction_1 = tf.to_int32(prediction_1)

# return result
result = sess.run(prediction_1)
No = np.arange(892,1310)
No = No.reshape(418,1)
title = np.array(["PassengerId","Survived"])
title = title.reshape(1,2)
result = np.append(No,result,axis=1)
result = np.append(title,result,axis=0)
np.savetxt("result.csv", result, delimiter=",",fmt = "%s")


In [42]:
sess.run(t_2)

array([[0.07008091],
       [0.39445674],
       [0.08345272],
       [0.08313822],
       [0.5156133 ],
       [0.19065274],
       [0.6529445 ],
       [0.14365527],
       [0.80493975],
       [0.06536259],
       [0.06928375],
       [0.29950097],
       [0.9511682 ],
       [0.04140777],
       [0.82907784],
       [0.9154984 ],
       [0.18714139],
       [0.22162239],
       [0.5225071 ],
       [0.6412065 ],
       [0.32015398],
       [0.2831989 ],
       [0.9455761 ],
       [0.5164421 ],
       [0.92547214],
       [0.02019735],
       [0.9677343 ],
       [0.20273708],
       [0.32705402],
       [0.06871321],
       [0.05891323],
       [0.11667173],
       [0.42634603],
       [0.44200724],
       [0.42071155],
       [0.256652  ],
       [0.570192  ],
       [0.61937714],
       [0.09434199],
       [0.08036573],
       [0.08237207],
       [0.3938969 ],
       [0.03859846],
       [0.8426815 ],
       [0.8440561 ],
       [0.09386905],
       [0.40245658],
       [0.090